# Exploratory Data Analysis on Sample Dataset


This notebook performs exploratory data analysis (EDA) on the sample dataset from the project. The goal is to understand the distribution of key variables, identify missing values, explore correlations, and investigate relationships between weather, energy mix, carbon intensity, and water intensity. Observations from previous analysis are included where relevant, with references to the figures and insights noted during earlier EDA (e.g. seasonal patterns【776473472252763†screenshot】, linear relationships between hydro share and water intensity【59990393566859†screenshot】, coal share vs. carbon intensity【810379427692406†screenshot】, and regional differences【265301069593362†screenshot】).


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load dataset (adjust the path if necessary)
file_path = '/home/oai/share/sofar_progress_unzip/sample_dataset.xlsx'
df = pd.read_excel(file_path)

# Display basic info
print('Dataset shape:', df.shape)
df.head()


In [ ]:

# Calculate missing value percentages
missing_pct = df.isnull().mean().sort_values(ascending=False) * 100
missing_pct = missing_pct[missing_pct > 0]
print('Missing value percentages (sorted descending):')
print(missing_pct)


In [ ]:

# Convert TIMESTAMP to pandas datetime (if not already)
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])


In [ ]:

# Plot histograms of carbon intensity and water intensity
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Histogram for kgco2_per_kwh (carbon intensity)
axs[0].hist(df['co2_kg'] / df['total_gen_kwh'], bins=30)
axs[0].set_title('Distribution of Carbon Intensity (kgCO$_2$ per kWh)')
axs[0].set_xlabel('kgCO$_2$ per kWh')
axs[0].set_ylabel('Frequency')

# Histogram for WUE_total (water intensity)
axs[1].hist(df['WUE_total'], bins=30)
axs[1].set_title('Distribution of Water Intensity (WUE\_total)')
axs[1].set_xlabel('WUE\_total (liters per kWh)')
axs[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()


In [ ]:

# Compute daily averages of carbon intensity and water intensity
df['date'] = df['TIMESTAMP'].dt.date

daily_df = df.groupby('date').agg({
    'co2_kg': 'sum',
    'total_gen_kwh': 'sum',
    'WUE_total': 'mean'
}).reset_index()
# Carbon intensity as daily total emissions divided by total generation

# Avoid division by zero
mask = daily_df['total_gen_kwh'] != 0
daily_df.loc[mask, 'kgco2_per_kwh'] = daily_df.loc[mask, 'co2_kg'] / daily_df.loc[mask, 'total_gen_kwh']

fig, axs = plt.subplots(2, 1, figsize=(12, 6), sharex=True)
axs[0].plot(daily_df['date'], daily_df['kgco2_per_kwh'])
axs[0].set_ylabel('kgCO$_2$ per kWh')
axs[0].set_title('Daily Average Carbon Intensity')

axs[1].plot(daily_df['date'], daily_df['WUE_total'])
axs[1].set_ylabel('WUE\_total (liters per kWh)')
axs[1].set_title('Daily Average Water Intensity')

plt.tight_layout()
plt.show()


In [ ]:

# Scatter plots of weather vs water intensity

# Temperature vs WUE_total
plt.figure(figsize=(6,4))
plt.scatter(df['temp_c'], df['WUE_total'], alpha=0.5)
plt.title('Temperature (°C) vs WUE\_total')
plt.xlabel('temp_c (°C)')
plt.ylabel('WUE\_total (liters per kWh)')
plt.show()

# Relative humidity vs WUE_total
plt.figure(figsize=(6,4))
plt.scatter(df['rh_pct'], df['WUE_total'], alpha=0.5)
plt.title('Relative Humidity (%) vs WUE\_total')
plt.xlabel('rh_pct (%)')
plt.ylabel('WUE\_total (liters per kWh)')
plt.show()

# Coal share vs carbon intensity
plt.figure(figsize=(6,4))
# Compute coal share for scatter plot
coal_share = df['COAL'] / df['total_gen_mwh']
kgco2_per_kwh = df['co2_kg'] / df['total_gen_kwh']
plt.scatter(coal_share, kgco2_per_kwh, alpha=0.5)
plt.title('Coal Share vs Carbon Intensity')
plt.xlabel('Coal Share')
plt.ylabel('kgCO$_2$ per kWh')
plt.show()


In [ ]:

# Correlation heatmap for fuel share percentages and intensities
fuel_cols = ['COAL', 'HYDRO', 'NATURALGAS', 'NUCLEAR', 'OTHER', 'PETROLEUM', 'SOLAR', 'WIND']
# Compute fuel shares
shares = pd.DataFrame()
for col in fuel_cols:
    shares[col + '_share'] = df[col] / df['total_gen_mwh']
# Add intensities
shares['kgco2_per_kwh'] = df['co2_kg'] / df['total_gen_kwh']
shares['WUE_total'] = df['WUE_total']

# Compute Spearman correlation
corr_matrix = shares.corr(method='spearman')

# Plot heatmap using matplotlib
plt.figure(figsize=(10,8))
cax = plt.imshow(corr_matrix, cmap='viridis', aspect='auto')
plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=90)
plt.yticks(range(len(corr_matrix.index)), corr_matrix.index)
plt.colorbar(cax)
plt.title('Spearman Correlation Heatmap (Fuel Shares & Intensities)')
plt.tight_layout()
plt.show()


In [ ]:

# Median WUE_total by EGRID region
region_medians = df.groupby('EGRIDREGION')['WUE_total'].median().sort_values(ascending=False)

plt.figure(figsize=(8,6))
plt.barh(region_medians.index, region_medians.values)
plt.xlabel('Median WUE\_total (liters per kWh)')
plt.ylabel('EGRIDREGION')
plt.title('Median WUE\_total by Region')
plt.gca().invert_yaxis()
plt.show()



### Observations

* **Seasonal variation:** Water intensity shows clear seasonal patterns, peaking in the spring and dipping during the winter months【776473472252763†screenshot】. Carbon intensity tends to drop during the spring/early summer and rise toward late summer and autumn.
* **Fuel mix effects:** Regions with higher hydro generation share (e.g., NWPP, NYCW, CAMX) display higher median water intensities【265301069593362†screenshot】. The scatter plot of hydro share vs water intensity from previous analysis confirmed a nearly linear positive relationship【59990393566859†screenshot】. Conversely, coal share is strongly correlated with carbon intensity【810379427692406†screenshot】.
* **Weather influence:** Higher air temperatures and lower relative humidity correspond to higher water intensity due to increased cooling load, while precipitation appears weakly related (not shown here but observed in past EDA).
* **Regional disparity:** Western and northeastern regions have significantly higher water usage per kWh compared to southeastern regions【265301069593362†screenshot】.

These insights inform which features and relationships to model in the predictive stage. We now proceed to prepare the data properly for modeling.
